# Simple Calculator using Reverse Polish Notation

[**Reverse Polish Notation (RPN)**](https://en.wikipedia.org/wiki/Reverse_Polish_notation), also known as **Polish Postfix Notation** or simply **Postfix Notation**, is a mathematical notation in which operators follow their operands, in contrast to **Polish Notation (PN)**, in which operators precede their operands. It does not need any parentheses as long as each operator has a fixed number of operands. The description **"Polish"** refers to the nationality of logician **Jan Łukasiewicz**, who invented **Polish notation** in 1924.<br/>

The **reverse Polish** scheme was proposed in 1954 by **Arthur Burks, Don Warren,** and **Jesse Wright** and was independently reinvented by **Friedrich L. Bauer** and **Edsger W. Dijkstra** in the early 1960s to reduce computer memory access and utilize the stack to evaluate expressions. The algorithms and notation for this scheme were extended by Australian philosopher and computer scientist **Charles L. Hamblin** in the mid-1950s.<br/>

**Postfix Notataion** remove all the parentheses and make the process of calculate expression much easier, especially for computer.<br/>

In order to understand **Postfix Notation**, we can use an [**Expression Tree**](https://en.wikipedia.org/wiki/Binary_expression_tree) to visualize this expression:
**-sqrt(36.2 - 5.3) + 2 - (3.1 + 5.52) * 4 / -3.0**

![Expression Tree](https://s2.upanh.pro/2018/08/09/tree.png)

That expression will be **"36.2 5.3 - sqrt - 2.0 + 3.1 5.52 + 4.0 * -3.0 / - -"** in **Postfix Notation**

In the next section, I will show you how we can convert an **Infix Expression** into a **Postfix Expression**, evaluate it and then plot a **Binary Expression Tree** like that.

## 1. Pre-processing and prepraring:
Our input will be a string, so I will remove any space, then add some necessary space and replace some special case like two minus character '--' to one plus character '+' to Normalize our expression. Finally, I used [**Regular Expression**](https://en.wikipedia.org/wiki/Regular_expression) to slice that string in to a list of operand and operator.<br/>

**Note:** We might not need to mess with string if we can use **Regular Expression** to solve all the pre-processing thing when slice the expression. I don't have strong experience in **Regex** so this is what I did for now.

In [1]:
import re


def preprocessExpression(string):
    string = string.replace(' ', '')
    string = string.replace(')-', ') - ')
    string = string.replace('*-', ' * -')
    string = string.replace('/-', ' / -')
    string = string.replace('+-', ' + -')
    string = string.replace('--', ' + ')
    i = 1
    if len(string) != 1:
        while True:
            while string[i] == '-' and string[i - 1].isdigit() and string[i + 1].isdigit():
                string = string[:i] + ' ' + string[i] + ' ' + string[i + 1:]
            i += 1
            if i == len(string):
                break
        operator  = ['+','-','*','/','^','sqrt']
        while True:
            while string[i] not in operator and string and string[i+1]
    exp = re.findall(r'(-*[0-9,\.]+)|([*+^\/-]+|[A-Za-z]+)|(\(|\))', string)
    exp = [tuple(j for j in i if j)[-1] for i in exp]
    for i, x in enumerate(exp):
        try:
            exp[i] = float(x)
        except ValueError:
            pass
    return exp

Let's try it out:

In [2]:
_input = "-sqrt(36.2 - 5.3) + 2 - (3.1 + 5.52) * 4 / -3.0"
exp = preprocessExpression(_input)
print(exp)

['-', 'sqrt', '(', 36.2, '-', 5.3, ')', '+', 2.0, '-', '(', 3.1, '+', 5.52, ')', '*', 4.0, '/', -3.0]


**Perfect!** Next, we have to determine which operator will have higher priority. In this calculator, I will use 6 operators: **+ - * / ^** and **sqrt**. We can add any operators like **sin, cos, log, ln, ...** easily down the lines:

In [3]:
def notGreater(i, j):
    precedence = {'+': 1, '-': 1, '*': 2, '/': 2, '^': 3, 'sqrt': 4}
    try:
        a = precedence[i]
        b = precedence[j]
        return True if a <= b else False
    except KeyError:
        return False

## 2. Convert Infix Expressions to Postfix Expressions:
We can convert **Infix Expressions** to **Postfix Expressions** by using [**Shunting-Yard Algorithm**](https://en.wikipedia.org/wiki/Shunting-yard_algorithm), **Edsger Dijkstra** invented it, so named because its operation resembles that of a railroad shunting yard.

Here is the **Shunting-Yard Algorithm**:
* **Step 1:** Scan the infix expression from left to right.
* **Step 2:** If the scanned character is an operand, output it.
* **Step 3:** Else,
    * If the precedence of the scanned operator is greater than the precedence of the operator in the stack(or the stack is empty), push it.
    * Else, Pop the operator from the stack until the precedence of the scanned operator is less-equal to the precedence of the operator residing on the top of the stack. Push the scanned operator to the stack.
* **Step 4:** If the scanned character is an ‘(‘, push it to the stack.
* **Step 5:** If the scanned character is an ‘)’, pop and output from the stack until an ‘(‘ is encountered.
* **Step 6:** Repeat steps 2-6 until infix expression is scanned.
* **Step 7:** Pop and output from the stack until it is not empty.

In [4]:
def infixToPostfix(exp):
    output = []
    stack = []
    for i in exp:
        if type(i) is float:
            output.append(i)
        elif i == '(':
            stack.append('(')
        elif i == ')':
            while stack and stack[-1] != '(':
                a = stack.pop()
                output.append(a)
            if stack and stack[-1] != '(':
                return -1
            else:
                stack.pop()
        else:
            while stack and notGreater(i, stack[-1]):
                output.append(stack.pop())
            stack.append(i)
    while stack:
        output.append(stack.pop())

    return output

Let's do a quick test:

In [5]:
postfix = infixToPostfix(exp)
print(postfix)

[36.2, 5.3, '-', 'sqrt', '-', 2.0, '+', 3.1, 5.52, '+', 4.0, '*', -3.0, '/', '-']


Because each token will be read once, each number, function, or operator will be printed once, and each function, operator, or parenthesis will be pushed onto the stack and popped off the stack once—therefore, there are at most a constant number of operations executed per token, and the running time is thus ***O(n)*** — linear in the size of the input.

# 3. Evaluate Prefix Expressions:
Now we have an **Prefix Expression**, we need to evaluate it for our calculators output:
* **Step 1:** Create a stack to store operands (or values).
* **Step 2:** Scan the given expression and do following for every scanned element.
    * If the element is a number, push it into the stack.
    * If the element is a operator, pop operands for the operator from stack. Evaluate the operator and push the result back to the stack.
* **Step 3:** When the expression is ended, the number in the stack is the final answer.

In [6]:
from math import sqrt

def evaluatePostfix(postfix):
    stack = []
    for i in postfix:
        if type(i) is float:
            stack.append(i)
        else:
            if i == 'sqrt':
                val = stack.pop()
                stack.append(sqrt(val))
            elif i == '-' and len(stack) == 1:
                val = stack.pop()
                stack.append(-val)
            else:
                val1 = stack.pop()
                val2 = stack.pop()
                if i == '+':
                    stack.append(val2 + val1)
                elif i == '-':
                    stack.append(val2 - val1)
                elif i == '*':
                    stack.append(val2 * val1)
                elif i == '/':
                    stack.append(val2 / val1)
                elif i == '^':
                    stack.append(val2**val1)

    return stack.pop()

In [7]:
res = evaluatePostfix(postfix)
print(res)

7.934556489458414


Time complexity of evaluation algorithm is ***O(n)*** where n is number of characters in input expression.

# 4. Plug everything in a simple calculator:
All the hard problems have been solved, it's time for our simple console based calculator, let's catch some error and we are good to go ^^

In [8]:
try:
    # Test expression: -sqrt(36.2 - 5.3) + 2 - (3.1 + 5.52) * 4 / -3.0
    expression = input("Input your expression:")
    postfix = infixToPostfix(preprocessExpression(expression))
    print("Postfix expression: ", end='')
    for each in postfix:
        print(str(each), ' ', end='')
    print("\nValue: ",evaluatePostfix(postfix))
except ValueError:
    print("Math Error!")
except IndexError:
    print("Expression Not Valid!")

Input your expression:-sqrt(36.2 - 5.3) + 2 - (3.1 + 5.52) * 4 / -3.0
Postfix expression: 36.2  5.3  -  sqrt  -  2.0  +  3.1  5.52  +  4.0  *  -3.0  /  -  
Value:  7.934556489458414


**And it's worked !**

![It's worked](http://sv1.upsieutoc.com/2018/08/09/Screen-Shot-2018-08-09-at-11.55.51.png)

# 5. Plot the Expression Tree:

In order to understand how everything works, we might need to use a **Binary Expression Tree** to visualize the expression. **Prefix Notation** help up construct the binary tree way easier than **Infix Notation**. The process of construct an **Expression Tree** is pretty much the same when we evaluate the **Prefix Expression**.

I used **[anytree](https://anytree.readthedocs.io/en/latest/intro.html)**, a simple and powerful Python **tree data library** to construct and plot our tree.

Since **Binary Expression Tree** is **undirected tree**, I have to make a **edege type function** make sure anytree output the correct tree:

In [9]:
def edgetypefunc(node, child):
    return '--'

Each node of the tree have **a key** in **anytree's Node Class**, so we **can't use a operand or operator** for that key since there might be are a lot of operand or operator got **dupicated** in an expression. But we still can save that value by a additional **kwargs**, so we have to tell anytree to render tree using that **kwargs**.

In [10]:
def nodeattrfunc(node):
    return 'label="' + str(node.val) + '"'

Here is how we will construct an **Expression Tree** from an **Prefix Expression**:
* **Step 1:** Create a stack to store node and an index for those node
* **Step 2:** Scan the prefix expression from left to right and do following for every scanned element.
    * If the element is a number, create a node for it with new index and push it to the stack
    * If the element is a operator:
        * Create a node for it with new index
        * Pop the operands for that operator from stack then set their parent to the operator node
        * Push the parent node to the stack

* At the end, there should be **only one node** left in the stack and it is the **root** node of our **Expression Tree**

In [11]:
from anytree import Node, RenderTree, DoubleStyle
from anytree.exporter import DotExporter


def drawExpressionTree(postfix):
    stack = []
    index = 0
    for i in postfix:
        if type(i) is float:
            stack.append(Node(index, parent=None, val=i))
            index += 1
        else:
            if i == 'sqrt':
                child = stack.pop()
                parent = Node(index, parent=None, val=i)
                index += 1
                child.parent = parent
                stack.append(parent)
            elif i == '-' and len(stack) == 1:
                child = stack.pop()
                parent = Node(index, parent=None, val=i)
                index += 1
                child.parent = parent
                stack.append(parent)
            else:
                right = stack.pop()
                left = stack.pop()
                parent = Node(index, parent=None, val=i)
                index += 1
                left.parent = parent
                right.parent = parent
                stack.append(parent)
    root = stack.pop()
    print(RenderTree(root, style=DoubleStyle).by_attr(attrname='val'))
    DotExporter(root, graph="graph", nodeattrfunc=nodeattrfunc,
                edgetypefunc=edgetypefunc).to_dotfile("tree.dot")

In this function, i used **RenderTree()** function to render our **Expression Tree** in console and **DotExporter** to export our tree to a ***.dot*** file, we can use dot tool from the [graphviz](http://www.graphviz.org/) package to render it into a ***.png*** image. For futher customization and decoration, please head to [Anytree's Documentation](https://anytree.readthedocs.io/en/latest/intro.html):

In [12]:
drawExpressionTree(postfix)

-
╠══ +
║   ╠══ -
║   ║   ╚══ sqrt
║   ║       ╚══ -
║   ║           ╠══ 36.2
║   ║           ╚══ 5.3
║   ╚══ 2.0
╚══ /
    ╠══ *
    ║   ╠══ +
    ║   ║   ╠══ 3.1
    ║   ║   ╚══ 5.52
    ║   ╚══ 4.0
    ╚══ -3.0


In [13]:
!dot tree.dot -T png -o tree.png

![Expression Tree](tree.png "Expression Tree")

You can also copy all content inside the **tree.dot** file and use [Webgprahviz](http://www.webgraphviz.com/) to plot the tree right inside your browser:

In [14]:
f = open('tree.dot', 'r')
file_contents = f.read()
print(file_contents)
f.close()

graph tree {
    "14" [label="-"];
    "6" [label="+"];
    "4" [label="-"];
    "3" [label="sqrt"];
    "2" [label="-"];
    "0" [label="36.2"];
    "1" [label="5.3"];
    "5" [label="2.0"];
    "13" [label="/"];
    "11" [label="*"];
    "9" [label="+"];
    "7" [label="3.1"];
    "8" [label="5.52"];
    "10" [label="4.0"];
    "12" [label="-3.0"];
    "14" -- "6";
    "14" -- "13";
    "6" -- "4";
    "6" -- "5";
    "4" -- "3";
    "3" -- "2";
    "2" -- "0";
    "2" -- "1";
    "13" -- "11";
    "13" -- "12";
    "11" -- "9";
    "11" -- "10";
    "9" -- "7";
    "9" -- "8";
}



<img src="https://media.giphy.com/media/2fsdaNR299EuyYtVJ7/giphy.gif">

**Note:** ***anytree*** support render tree to picture with just a simple line of code but it have trouble working with anaconda so i didn't use it. Also, this project is **not prefect**. There are still **a lot of room for improvement** like **better regex for slicing expression, evaluate expression using expression tree and DFS, etc..**

#### ***Anyways, thanks for read this till the end! Feels free to give me any question or contribute to this project <3***

In [8]:
string = '456(3+6)'
i = 2
string = string[:i+1] + '*' + string[i+1:]
print(string)

456*(3+6)


In [9]:
list = [3,4,5,6]
print(list[:-1])

[3, 4, 5]


In [10]:
postfix = [3.0, 4.0, '+']
for each in postfix:
        print(str(each), ' ', end='')

3.0  4.0  +  

In [13]:
import re
s = "(2+3)(5+4)8"
replaced = re.sub(r'(\))(\(|[0-9])', r'\1*\2', s)
print (replaced)

(2+3)*(5+4)*8


In [16]:
try:
    3/0
except ZeroDivisionError:
    print("OK")

OK
